In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




In [4]:
# Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
Total = np.unique(Breath_dataframe_0.iloc[:,73].values)
name = []
for i in Total:
    name.append(i[:4])
unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
NAsthma = []
for file in unique:
    if file.split("_")[2] == 'Asthma' or file.split("_")[2] == 'asthma' :
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.split("_")[2] == 'NAsthma':
        NAsthma.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [5]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")
print(f"NAsthma = {len(NAsthma)}")

Total = 274
Asthmatic_male = 46
Asthmatic_female = 28
Healthy_Male = 55
Healthy_female = 51
Before_after = 84
NAsthma = 10


In [3]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
#         ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier(max_depth = 8,
                     subsample = 0.8,
                     learning_rate = 0.01,
                     n_estimators = 450,
                     min_child_weight = 1)
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [3]:
def make_legit_sets(sound_dataframe, sound, path):
    for i in range(0,10):
        Train = np.loadtxt(f"./{path}/Train/Train_{i}.txt", dtype= 'str')
        Test = np.loadtxt(f"./{path}/Test/Test_{i}.txt", dtype= 'str')


        test_dataframe = pd.DataFrame()
        for j, line in enumerate(Test):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[j]))]
            test_dataframe = pd.DataFrame.append(test_dataframe,A)

        train_dataframe = pd.DataFrame()
        for j, line in enumerate(Train):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[j]))]
            train_dataframe = pd.DataFrame.append(train_dataframe,A)
#################################################
        train_dataframe.to_csv('./breathe_set_csv_exp_5/train'+sound+str(i)+'.csv', index=False)
        test_dataframe.to_csv('./breathe_set_csv_exp_5/test'+sound+str(i)+'.csv', index=False)
    return None

In [6]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [9]:
make_legit_sets(Wheeze_dataframe, 'Wheeze', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_1, 'Wheeze_1', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_2, 'Wheeze_2', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_3, 'Wheeze_3', "Wheeze_sets")

In [75]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv_exp_4/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv_exp_4/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv_exp_4/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:19:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [76]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8445378151260504, 0.890...",0.862794,0.0401354
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8697478991596639, 0.898...",0.857496,0.0493695
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9721577726218097, 0.97, 0.9664586583463338,...",0.972642,"[0.8979591836734694, 0.8151260504201681, 0.843...",0.815662,0.0482381
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8894044856921887, 0.9023076923076923, 0.883...",0.898692,"[0.8938775510204081, 0.7521008403361344, 0.839...",0.807796,0.0517178


In [77]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8571428571428571, 0.7647058823529411, 0.828...",0.784965,0.045702
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8285714285714286, 0.7815126050420168, 0.824...",0.78456,0.0395398
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8507347254447022, 0.8584615384615385, 0.851...",0.861377,"[0.8244897959183674, 0.7521008403361344, 0.859...",0.782321,0.0444023
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9303944315545244, 0.9276923076923077, 0.926...",0.932318,"[0.8081632653061225, 0.7563025210084033, 0.816...",0.779208,0.0401786


In [78]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8361344537815126, 0.875...",0.851052,0.0435679
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8529411764705882, 0.875...",0.845398,0.0509733
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8923076923076924, 0.8923...",0.891199,"[0.889795918367347, 0.7605042016806722, 0.8125...",0.806633,0.0477326


In [79]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8646558391337974, 0.8784615384615385, 0.867...",0.872846,"[0.8857142857142857, 0.8025210084033614, 0.828...",0.803741,0.0384212
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.946923076923077, 0.9446...",0.946774,"[0.889795918367347, 0.7983193277310925, 0.7812...",0.798646,0.0367506
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.7773109243697479, 0.824...",0.794951,0.0451413
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8408163265306122, 0.773109243697479, 0.8320...",0.792802,0.0358028


In [60]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")

In [61]:
make_legit_sets(Wheeze_dataframe_0, 'Wheeze_0', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_5, 'Wheeze_5', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_10, 'Wheeze_10', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_15, 'Wheeze_15', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_20, 'Wheeze_20', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_25, 'Wheeze_25', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_30, 'Wheeze_30', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_35, 'Wheeze_35', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_40, 'Wheeze_40', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_45, 'Wheeze_45', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_50, 'Wheeze_50', "Wheeze_sets")

In [62]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv_exp_4/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv_exp_4/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv_exp_4/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv_exp_4/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv_exp_4/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv_exp_4/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv_exp_4/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv_exp_4/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv_exp_4/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv_exp_4/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:50:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:58:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:58:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [63]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.8067226890756303, 0.878...",0.825883,0.0498517
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.7941176470588235, 0.867...",0.824202,0.0438369
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.9461538461538461, 0.944...",0.95146,"[0.8857142857142857, 0.8025210084033614, 0.820...",0.811478,0.0490403
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.877030162412993, 0.8838461538461538, 0.8728...",0.885553,"[0.9061224489795918, 0.7773109243697479, 0.859...",0.797247,0.0590706


In [64]:
MLA_wheeze_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.7899159663865546, 0.851...",0.8227,0.052671
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8025210084033614, 0.859...",0.822147,0.0534777
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9466357308584686, 0.9484615384615385, 0.948...",0.951981,"[0.8816326530612245, 0.8277310924369747, 0.824...",0.811212,0.0552687
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.89, 0.8806552262090483,...",0.893351,"[0.9102040816326531, 0.8109243697478992, 0.851...",0.808739,0.0588626


In [65]:
MLA_wheeze_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999845,"[0.9183673469387755, 0.8067226890756303, 0.847...",0.826877,0.0522997
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.945862335653519, 0.9476923076923077, 0.9492...",0.951909,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.825491,0.0520221
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.7983193277310925, 0.875...",0.82279,0.0623617
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8863109048723898, 0.8838461538461538, 0.876...",0.888271,"[0.8857142857142857, 0.7352941176470589, 0.859...",0.79629,0.0592915


In [66]:
MLA_wheeze_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9346938775510204, 0.8319327731092437, 0.875...",0.842104,0.0498777
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.8151260504201681, 0.855...",0.838738,0.0476872
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9559164733178654, 0.95, 0.9492979719188768,...",0.957191,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.824198,0.0567637
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.8923076923076924, 0.882...",0.892286,"[0.889795918367347, 0.7563025210084033, 0.8398...",0.808507,0.0559755


In [67]:
MLA_wheeze_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9306122448979591, 0.8487394957983193, 0.875...",0.845931,0.0498238
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8529411764705882, 0.875...",0.845395,0.0520998
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9551430781129157, 0.9507692307692308, 0.956...",0.960132,"[0.9224489795918367, 0.8697478991596639, 0.863...",0.834722,0.0555252
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8824439288476411, 0.8953846153846153, 0.884...",0.889604,"[0.9061224489795918, 0.7647058823529411, 0.843...",0.817977,0.0452526


In [68]:
MLA_wheeze_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8403361344537815, 0.867...",0.844822,0.0493536
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8445378151260504, 0.859...",0.842665,0.0478151
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8915384615384615, 0.892...",0.891205,"[0.8857142857142857, 0.7563025210084033, 0.812...",0.804974,0.0477323


In [69]:
MLA_wheeze_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9387755102040817, 0.8445378151260504, 0.835...",0.84747,0.0465912
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9183673469387755, 0.819327731092437, 0.8476...",0.832815,0.0461284
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.96, 0.9648985959438378,...",0.96709,"[0.926530612244898, 0.8151260504201681, 0.8398...",0.829502,0.047383
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8923076923076924, 0.886...",0.893852,"[0.9020408163265307, 0.7815126050420168, 0.789...",0.819318,0.0364086


In [70]:
MLA_wheeze_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8487394957983193, 0.820...",0.825188,0.0448311
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.9646153846153847, 0.965...",0.965704,"[0.9061224489795918, 0.8277310924369747, 0.824...",0.823919,0.0418663
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8277310924369747, 0.816...",0.821434,0.0490248
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8869230769230769, 0.8822...",0.889634,"[0.8857142857142857, 0.7815126050420168, 0.777...",0.805244,0.0354808


In [71]:
MLA_wheeze_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8571428571428571, 0.820...",0.821839,0.0467198
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9613302397525135, 0.9607692307692308, 0.957...",0.963138,"[0.9020408163265307, 0.8151260504201681, 0.820...",0.817164,0.044771
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992266047950503, 1.0, 0.999219968798752, 1...",0.999615,"[0.8979591836734694, 0.8403361344537815, 0.816...",0.814955,0.0507342
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8747099767981439, 0.8838461538461538, 0.875...",0.886102,"[0.8816326530612245, 0.7815126050420168, 0.761...",0.786197,0.0406657


In [72]:
MLA_wheeze_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8361344537815126, 0.808...",0.819029,0.040269
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.999219968798752, 0.99921259842519...",0.999766,"[0.8816326530612245, 0.8361344537815126, 0.820...",0.805194,0.0512598
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8669760247486465, 0.8738461538461538, 0.870...",0.876507,"[0.889795918367347, 0.773109243697479, 0.76171...",0.794319,0.041029
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.9592307692307692, 0.955...",0.962747,"[0.889795918367347, 0.7941176470588235, 0.8046...",0.792668,0.0481288


In [73]:
MLA_wheeze_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.8319327731092437, 0.785...",0.80965,0.0375153
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992307692307693, 0.999219968798752, 0...",0.999538,"[0.889795918367347, 0.8277310924369747, 0.8320...",0.804897,0.0535399
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8739365815931941, 0.8853846153846154, 0.872...",0.882987,"[0.8734693877551021, 0.7563025210084033, 0.777...",0.797445,0.0407303
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.96, 0.9609984399375975,...",0.963401,"[0.8734693877551021, 0.8109243697478992, 0.796...",0.79547,0.0406523


In [4]:
breath_dataframe = pd.read_csv("./breath_csv_2//Breath.csv")
breath_dataframe_1 = pd.read_csv("./breath_csv_2//Breath_1.csv")
breath_dataframe_2 = pd.read_csv("./breath_csv_2//Breath_2.csv")
breath_dataframe_3 = pd.read_csv("./breath_csv_2//Breath_3.csv")

In [5]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets")

In [6]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv_exp_5/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv_exp_5/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv_exp_5/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv_exp_5/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[09:57:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:11] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[10:02:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [7]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.812, 0.7213740458015268, 0.8185483870967742...",0.766754,0.0704297
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 1.0, 0.9990453460620525, ...",0.999569,"[0.828, 0.6870229007633588, 0.8346774193548387...",0.766674,0.0779409
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9627329192546584, 0.9639596347909659, 0.960...",0.961303,"[0.752, 0.6793893129770993, 0.7459677419354839...",0.733473,0.0657154
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8413760152890588, 0.8447861604997597, 0.833...",0.840872,"[0.744, 0.6564885496183206, 0.7661290322580645...",0.725567,0.0773419


In [8]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.808, 0.7061068702290076, 0.6854838709677419...",0.730445,0.0686485
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9111323459149546, 0.9144641999038924, 0.912...",0.913944,"[0.772, 0.7137404580152672, 0.7016129032258065...",0.724779,0.0584042
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9995226730310263, ...",0.999331,"[0.768, 0.6793893129770993, 0.6814516129032258...",0.72346,0.062938
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7916865742952699, 0.7972128784238347, 0.806...",0.795913,"[0.736, 0.648854961832061, 0.625, 0.7792207792...",0.696639,0.0755518


In [9]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 1.0, ...",0.999952,"[0.812, 0.683206106870229, 0.8145161290322581,...",0.739855,0.0957061
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.82, 0.6946564885496184, 0.7903225806451613,...",0.739023,0.0929201
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9421882465360726, 0.9432964920711197, 0.936...",0.940972,"[0.788, 0.6984732824427481, 0.8306451612903226...",0.730691,0.0807408
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8275203057811753, 0.8399807784718886, 0.827...",0.833895,"[0.792, 0.6717557251908397, 0.7580645161290323...",0.730318,0.0861963


In [10]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8279980888676541, 0.8370975492551658, 0.823...",0.829445,"[0.804, 0.683206106870229, 0.8266129032258065,...",0.777279,0.0766033
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9216435738174868, 0.9308024987986545, 0.921...",0.924285,"[0.768, 0.6870229007633588, 0.8185483870967742...",0.766023,0.0679336
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990444338270426, 0.9990389235944258, 0.999...",0.99933,"[0.804, 0.6450381679389313, 0.7903225806451613...",0.74982,0.0824955
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.816, 0.6603053435114504, 0.8024193548387096...",0.748347,0.071622


Breath 10

In [19]:
for i in range(0,51,5):
    print(f"MLA_breath_{i} = MLA_selection(Breath_dataframe_{i}, 'breath_{i}', './breathe_set_csv_exp_5/') ")

MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_5/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_5/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_5/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_5/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_5/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_5/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_5/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_5/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_5/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_5/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_s

In [2]:
Breath_dataframe_0 = pd.read_csv('./breath_csv_2/Breath_0.csv')
Breath_dataframe_5 = pd.read_csv('./breath_csv_2/Breath_5.csv')
Breath_dataframe_10 = pd.read_csv('./breath_csv_2/Breath_10.csv')
Breath_dataframe_15 = pd.read_csv('./breath_csv_2/Breath_15.csv')
Breath_dataframe_20 = pd.read_csv('./breath_csv_2/Breath_20.csv')
Breath_dataframe_25 = pd.read_csv('./breath_csv_2/Breath_25.csv')
Breath_dataframe_30 = pd.read_csv('./breath_csv_2/Breath_30.csv')
Breath_dataframe_35 = pd.read_csv('./breath_csv_2/Breath_35.csv')
Breath_dataframe_40 = pd.read_csv('./breath_csv_2/Breath_40.csv')
Breath_dataframe_45 = pd.read_csv('./breath_csv_2/Breath_45.csv')
Breath_dataframe_50 = pd.read_csv('./breath_csv_2/Breath_50.csv')


In [16]:
make_legit_sets(Breath_dataframe_0, 'breath_0', 'breath_sets')
make_legit_sets(Breath_dataframe_5, 'breath_5', 'breath_sets')
make_legit_sets(Breath_dataframe_10, 'breath_10', 'breath_sets')
make_legit_sets(Breath_dataframe_15, 'breath_15', 'breath_sets')
make_legit_sets(Breath_dataframe_20, 'breath_20', 'breath_sets')
make_legit_sets(Breath_dataframe_25, 'breath_25', 'breath_sets')
make_legit_sets(Breath_dataframe_30, 'breath_30', 'breath_sets')
make_legit_sets(Breath_dataframe_35, 'breath_35', 'breath_sets')
make_legit_sets(Breath_dataframe_40, 'breath_40', 'breath_sets')
make_legit_sets(Breath_dataframe_45, 'breath_45', 'breath_sets')
make_legit_sets(Breath_dataframe_50, 'breath_50', 'breath_sets')

In [6]:
MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_5/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_5/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_5/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_5/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_5/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_5/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_5/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_5/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_5/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_5/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_set_csv_exp_5/') 


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[13:58:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[14:17:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [7]:
MLA_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 0.999...",0.999857,"[0.8, 0.7061068702290076, 0.8024193548387096, ...",0.752204,0.0750981
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.792, 0.7213740458015268, 0.7540322580645161...",0.745117,0.0674417
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.924988055422838, 0.9298414223930802, 0.9231...",0.927585,"[0.784, 0.6908396946564885, 0.7620967741935484...",0.725607,0.0670972
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8193979933110368, 0.8164344065353195, 0.806...",0.81448,"[0.74, 0.6526717557251909, 0.6895161290322581,...",0.692194,0.0810505


In [8]:
MLA_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 1.0, ...",0.999569,"[0.792, 0.7099236641221374, 0.8225806451612904...",0.749377,0.0779257
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.784, 0.7251908396946565, 0.7782258064516129...",0.747547,0.0672426
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9216435738174868, 0.932724651609803, 0.9212...",0.927014,"[0.772, 0.6679389312977099, 0.7580645161290323...",0.712978,0.0737255
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8217869087434305, 0.8130706391158097, 0.805...",0.816298,"[0.756, 0.6679389312977099, 0.7379032258064516...",0.696875,0.0844644


In [9]:
MLA_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.784, 0.7175572519083969, 0.7983870967741935...",0.742004,0.0793638
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 1.0, ...",0.999809,"[0.732, 0.7099236641221374, 0.8185483870967742...",0.738188,0.0835424
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9273769708552317, 0.9346468044209515, 0.925...",0.931366,"[0.764, 0.683206106870229, 0.7338709677419355,...",0.71115,0.0677753
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8184424271380794, 0.8207592503604036, 0.808...",0.819988,"[0.744, 0.6755725190839694, 0.717741935483871,...",0.693253,0.088087


In [10]:
MLA_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.796, 0.7290076335877863, 0.8064516129032258...",0.744487,0.0868747
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 1.0, 1.0, 1.0, 1.0, 0.999...",0.999856,"[0.78, 0.7175572519083969, 0.8387096774193549,...",0.742018,0.087782
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9345437171524128, 0.9375300336376742, 0.930...",0.93472,"[0.776, 0.6641221374045801, 0.7661290322580645...",0.72147,0.0708638
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8198757763975155, 0.8241230177799135, 0.808...",0.822711,"[0.748, 0.6564885496183206, 0.7540322580645161...",0.699132,0.0841056


In [11]:
MLA_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.792, 0.7175572519083969, 0.8185483870967742...",0.74363,0.0910904
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.7213740458015268, 0.8185483870967742...",0.740879,0.0907456
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9426660296225514, 0.9413743392599712, 0.936...",0.939921,"[0.78, 0.7099236641221374, 0.7943548387096774,...",0.730458,0.0737238
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8284758719541329, 0.8366170110523786, 0.809...",0.828741,"[0.76, 0.6679389312977099, 0.7701612903225806,...",0.716482,0.088979


In [12]:
MLA_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 1.0, ...",0.999952,"[0.812, 0.683206106870229, 0.8145161290322581,...",0.739855,0.0957061
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.683206106870229, 0.8024193548387096,...",0.734816,0.0856085
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9421882465360726, 0.9432964920711197, 0.936...",0.940972,"[0.788, 0.6984732824427481, 0.8306451612903226...",0.730691,0.0807408
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8275203057811753, 0.8399807784718886, 0.825...",0.833277,"[0.792, 0.6717557251908397, 0.7580645161290323...",0.730469,0.088623


In [13]:
MLA_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.816, 0.6946564885496184, 0.8064516129032258...",0.746336,0.0879082
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9995226730310263, ...",0.999904,"[0.792, 0.6755725190839694, 0.8024193548387096...",0.746075,0.0942234
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.94696607740086, 0.9442575684766938, 0.94224...",0.943172,"[0.784, 0.7022900763358778, 0.8266129032258065...",0.734008,0.0808948
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8322981366459627, 0.8457472369053339, 0.825...",0.836864,"[0.76, 0.648854961832061, 0.7782258064516129, ...",0.73039,0.0899813


In [14]:
MLA_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 0.999...",0.999568,"[0.788, 0.6870229007633588, 0.7741935483870968...",0.742609,0.0853906
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.6679389312977099, 0.8145161290322581, ...",0.736114,0.0939631
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9474438604873388, 0.9471407976934166, 0.943...",0.942603,"[0.756, 0.6526717557251909, 0.8064516129032258...",0.728837,0.0790274
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8327759197324415, 0.8423834694858241, 0.824...",0.836532,"[0.74, 0.6412213740458015, 0.7862903225806451,...",0.72251,0.0827679


In [15]:
MLA_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9990453460620525, ...",0.999568,"[0.804, 0.6755725190839694, 0.782258064516129,...",0.753695,0.0880127
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.832, 0.6679389312977099, 0.8024193548387096...",0.748831,0.0871562
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9488772097467749, 0.948582412301778, 0.9441...",0.944666,"[0.76, 0.6793893129770993, 0.8266129032258065,...",0.739708,0.0755281
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8256091734352604, 0.8385391638635271, 0.820...",0.828835,"[0.748, 0.648854961832061, 0.7983870967741935,...",0.721248,0.0872215


In [16]:
MLA_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990444338270426, 0.9995194617972128, 0.998...",0.999234,"[0.796, 0.6450381679389313, 0.8064516129032258...",0.749028,0.097042
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.6603053435114504, 0.8024193548387096, ...",0.747322,0.0741989
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8198757763975155, 0.8395002402691014, 0.820...",0.831349,"[0.784, 0.6374045801526718, 0.7862903225806451...",0.732425,0.086569
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9507883420926899, 0.952426717924075, 0.9451...",0.946103,"[0.768, 0.6412213740458015, 0.7782258064516129...",0.726897,0.0617964


In [17]:
MLA_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9990389235944258, 0.9990453460620525, ...",0.999474,"[0.82, 0.6641221374045801, 0.8185483870967742,...",0.756984,0.0988959
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.6679389312977099, 0.8024193548387096...",0.751713,0.0789307
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8303870043000477, 0.8515136953387794, 0.829...",0.838003,"[0.772, 0.6259541984732825, 0.7983870967741935...",0.744417,0.0837044
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9498327759197325, 0.9490629505045651, 0.949...",0.947532,"[0.76, 0.6450381679389313, 0.782258064516129, ...",0.731446,0.064504
